In [1]:
import numpy as np
import nltk
from collections import defaultdict
from src.classes.database.offer import MongoOffer
from tqdm import tqdm
from enum import Enum

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [2]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
direction = Enum('direction', ['FORWARD', 'BACKWARD', 'SELF'])

In [3]:
def get_all_words(token_list: list[list]) -> list:
    words = []
    for sentence in token_list:
        for word in sentence:
            words.append(word)
    return sorted(set(words))

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

# Old method, slow and not used anymore
def calculate_all_probabilities(tokens: list[list], look_ahead) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = defaultdict()
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [defaultdict(float) for x in range(0,look_ahead)]
        for x in range(0,look_ahead):
            for sub_word in all_words:
                p = p_next_word(word, sub_word, x+1, tokens)
                if p > 0:
                    p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")
    return p_word
#data = calculate_all_probabilities(token_list, 1)

## HMaxtrix
Used for storing all word occurrences in a 3 dimensional matrix

### Data we can get from matrix
 1. p(start, find) = probability that 'find' is a n-order forward word for 'start' | single cell value / sum of 'start' row
 2. p(start, find) = probability that 'find' is a n-order backward word for 'start' | single cell value / sum of 'start' column
 3. p(row_word) = probability that 'row word' will be a n-order forward word | sum of row / entire table sum
 4. p(col_word) = probability that 'col word' will be a n-order backward word | sum of column / entire table sum
 5. p(word, order) = probability that 'word' will be n-order word compared to the other orders| single cell value at order / all order values of that word tallied (through the table)
 6.  p(word-row, order) = probability that 'word' will be n-order forward word | all of word-row order sum / all order values of that word-row
 7. p(col-row, order) = probability that 'word' will be n-order backward word | all of word-col order sum / all order values of that word-col

In [10]:
class HMatrix:
    def __init__(self, verbose=False) -> None:
        self.labels = None
        self.reverse_labels = None
        self.order = 0
        self.matrix = None
        self.verbose = verbose

    def p_row_word(self, order: int, row_word: str, word: str) -> float:
        if row_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        row_label = self.reverse_labels[row_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, row_label, :][word_label]
        divisor = sum(self.matrix[order, row_label, :])

        return dividend/divisor if divisor > 0 else 0

    def p_col_word(self, order: int, col_word: str, word: str) -> float:
        if col_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        col_label = self.reverse_labels[col_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, :, col_label][word_label]
        divisor = sum(self.matrix[order, :, col_label])

        return dividend/divisor if divisor > 0 else 0


    def create_matrix(self, tokens: list[list], order: int):
        #Setup
        self.labels = get_all_words(token_list)
        self.reverse_labels = {self.labels[x]: x for x in range(0, len(self.labels))}
        self.order = order
        self.matrix = np.zeros((order, len(self.labels), len(self.labels)))

        #Iteration
        for sentence in tqdm(tokens):
            for x in range(0, len(sentence)):
                word = sentence[x]
                if self.verbose:
                    print(f"Word: {word} ({str(self.reverse_labels[word])})")
                for y in range(0, self.order):
                    if x+y+1 < len(sentence):
                        if self.verbose:
                            print(f"Lookahead {str(y+1)}: {sentence[x+y+1]} ({str(self.reverse_labels[sentence[x+y+1]])})")
                        self.matrix[y][self.reverse_labels[word]][self.reverse_labels[sentence[x+y+1]]] += 1

In [12]:
matrix = HMatrix(verbose=False)
matrix.create_matrix(token_list, 3) #todo, juypter seems to break when order > 3

100%|██████████| 5416/5416 [00:00<00:00, 56545.47it/s]


## Sentence Class

Input a sentence, it will then use the HMatrix to see which path results the most likely outcome for products

In [7]:
class Sentence:
    def __init__(self, matrix: HMatrix, tokenizer, raw_sentence: str, verbose=False):
        self.verbose = verbose
        self.raw_sentence = tokenizer.tokenize(raw_sentence.lower())
        self.sentence = []

        # Creating all the blank word classes
        for x, word in enumerate(self.raw_sentence):
            self.sentence.append(Word(word, x, matrix.order))

        # Creating all the forward connections
        for x, word in enumerate(self.raw_sentence):
            if verbose:
                print(f"--- {word} ---")
            for neighbor in self.sentence:
                mapped_pos = self.map_word_pos_to_order(neighbor.position-x)

                if mapped_pos[1] is not direction.SELF and mapped_pos[0] < matrix.order: #Looking at itself and order is within scope
                    p_value = 0

                    if mapped_pos[1] is direction.FORWARD:
                        p_value = matrix.p_row_word(mapped_pos[0], word, neighbor.key)
                    else:
                        p_value = matrix.p_col_word(mapped_pos[0], word, neighbor.key)


                    self.sentence[x].neighbors.append(
                            {'word': neighbor.key,
                             'ref': neighbor,
                             'p': p_value,
                             'direction': mapped_pos[1].name
                            })

                    if verbose:
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"-{str(mapped_pos[0])}-> "
                              f"{neighbor.key} "
                              f"({str(p_value)})")



    def map_word_pos_to_order(self, position) -> tuple:
        #Forward word positions
        if position > 0:
            return position-1, direction.FORWARD

        # Backwards word positions
        if position < 0:
            return abs(position)-1, direction.BACKWARD

        #Pos looking at self
        if position == 0:
            return -1, direction.SELF



class Word:
    def __init__(self, key: str, position: int, order: int):
        self.key = key
        self.position = position
        self.matrix_order = order
        self.neighbors = []

    # This method will find this word's neighbor by taking the words position + an input position, returns None if out of bounds.
    def get_neighbor(self, pos):
        relative_pos = None
        if pos == 0:
            return None

        if pos > self.matrix_order: #cant look past an order we dont have saved
            return None

        if pos > 0: #positive lookahead
            #if self.position + pos - (self.matrix_order-1)> (self.matrix_order*2+1): #todo these bounds are different
            #if len(self.neighbors) < self.matrix_order*2 and len(self.neighbors) < (self.matrix_order + pos):
            if (self.position > self.matrix_order and len(self.neighbors) < (self.matrix_order + pos)) or len(self.neighbors) == 0:
                relative_pos = None
            else:
                if self.position < self.matrix_order:
                    relative_pos = self.position + (pos-1)
                else:
                    relative_pos = self.matrix_order + (pos-1)
        else: #negative lookahead
            if abs(pos) > self.position:
                relative_pos = None
            else:
                if self.position <= self.matrix_order-1:
                    relative_pos = (self.matrix_order - abs(pos) + (self.position - self.matrix_order))
                else:
                    relative_pos = self.matrix_order - abs(pos)

        if relative_pos is None:
            return None
        else:
            return self.neighbors[relative_pos]


    def __str__(self):
        return f"Word: {self.key} | Position: {str(self.position)}"

In [16]:
test = Sentence(matrix, tokenizer, "logitech g pro x superlight intel i7 8700k", verbose=False) #logitech g pro x superlight intel i7 8700k

In [19]:
# Use this method to test neighbors and see if they are correct
for o in range(-1*matrix.order, matrix.order+1):
    print(f"-- Order: {str(o)} --")
    for x in range(0, len(test.sentence)):
        testdata = test.sentence[x].get_neighbor(o)
        print(f"{test.sentence[x].key} -> {testdata['word'] if testdata is not None else None}")

-- Order: -3 --
logitech -> None
g -> None
pro -> None
x -> logitech
superlight -> g
intel -> pro
i7 -> x
8700k -> superlight
-- Order: -2 --
logitech -> None
g -> None
pro -> logitech
x -> g
superlight -> pro
intel -> x
i7 -> superlight
8700k -> intel
-- Order: -1 --
logitech -> None
g -> logitech
pro -> g
x -> pro
superlight -> x
intel -> superlight
i7 -> intel
8700k -> i7
-- Order: 0 --
logitech -> None
g -> None
pro -> None
x -> None
superlight -> None
intel -> None
i7 -> None
8700k -> None
-- Order: 1 --
logitech -> g
g -> pro
pro -> x
x -> superlight
superlight -> intel
intel -> i7
i7 -> 8700k
8700k -> None
-- Order: 2 --
logitech -> pro
g -> x
pro -> superlight
x -> intel
superlight -> i7
intel -> 8700k
i7 -> None
8700k -> None
-- Order: 3 --
logitech -> x
g -> superlight
pro -> intel
x -> i7
superlight -> 8700k
intel -> None
i7 -> None
8700k -> None


In [ ]:
def seperate_products(start_word: Word):
    products = []

# A method to look ahead only 1 word until the next word probability is 0
#todo not working
def traverse_to_first_order_end(word: Word):
    if word.get_neighbor(1) is None:
        return word

    print(f"Start: {word.key}")
    neighbor = word.get_neighbor(1)

    while neighbor['ref'].get_neighbor(1) is not None and neighbor['ref'].get_neighbor(1)['p'] > 0:
        print(f"Visited: {neighbor['word']}({str(neighbor['p'])})")

        # Go to next neighbor
        neighbor = neighbor['ref'].get_neighbor(1)

    if neighbor['ref'].get_neighbor(1) is None:
        print(f"EoS: {neighbor['word']}") # if at end of sentence
    else:
        print(f"EoP: {neighbor['word']}")# if at end of product


# Vector Method

In [20]:
class Vector:
    def __init__(self, items=None):
        self.items = items if items is not None else []

    def add(self, item, vector):
        if len(self.items) == 0:
            self.items.append({'items': [item], 'current_vector': vector})
        else:
            if self.items[-1]['current_vector'] == "r": #if current vector is right we dont care about the direction
                self.items[-1]['items'].append(item)
                self.items[-1]['current_vector'] = vector
            elif self.items[-1]['current_vector'] == "l":
                if vector == 'r': # new item
                    self.items.append({'items': [item], 'current_vector': vector})
                elif vector == 'l': # old item
                    self.items[-1]['items'].append(item)
                    self.items[-1]['current_vector'] = vector

In [23]:
# Take in a sentence, determine if a word should point right or left, then
def vector_assign(sentence: list[Word]):
    if len(sentence) == 1:
        return Vector([sentence[0].key])
    v = Vector()
    for x, word in enumerate(sentence):
        if x == 0: #First word always points right
            v.add(word.key, "r")
            #print(f"{word.key} -> {sentence[x].get_neighbor(1)['word']}")
        elif x == len(sentence)-1: #end of sentence
            v.add(word.key, "l")
            #print(f"{sentence[x].get_neighbor(-1)['word']} <- {word.key}")
        else:
            #print(f"Comparing: -1: {str(sentence[x].get_neighbor(-1)['p'])} | 1: {str(sentence[x].get_neighbor(1)['p'])}")
            if sentence[x].get_neighbor(1)['p'] >= sentence[x].get_neighbor(-1)['p']:
                v.add(word.key, "r")
                #print(f"{word.key}-> {sentence[x].get_neighbor(1)['word']}")
            else:
                v.add(word.key, "l")
                #print(f"{sentence[x].get_neighbor(-1)['word']} <- {word.key}")
    return v

In [24]:
vector_assign(test.sentence).items

[{'items': ['logitech', 'g', 'pro', 'x', 'superlight'], 'current_vector': 'l'},
 {'items': ['intel', 'i7', '8700k'], 'current_vector': 'l'}]

In [ ]:
# Stress Testing The Vector method among all words
count = 0
for word in token_list[0:100]:
    s = Sentence(matrix, tokenizer, " ".join(word), verbose=False)
    print("---")
    print(count)
    print(" ".join(word))
    print(vector_assign(s.sentence).items)
    print("---\n")
    count += 1


In [ ]:
vector_assign(test.sentence).items

In [ ]:
traverse_to_first_order_end(test.sentence[0])

From word a to word b (order 0) there are the following outcomes:
* p(a,b) > 0.0 has 2 outcomes
    * b is part of a
    * b is NOT part of a, but has been seen in the training data
* p(a,b) = 0.0 has 2 outcomes
    * b is NOT part of a
    * b is a new word
        * b is part of a
        * b is part of post b

# Attempting to use Naive Bayes to classify sentences


|   | intel | i7  | 8700k |
|-------|-------|-----|-------|
| intel | 0     | 1   | 0     |
| i7    | 0     | 0   | 1     |
| 8700k | 0     | 0   | 0     |

P(W_0 | W) = Given the first word is 'intel', what is the probability the word after it will be 'i7'
P(W) = probability that a given word is 'intel' =
P(W_0) = probability that the next word is 'i7' =
P(W | W_0) = given that the next word is 'i7', what is the probability that the first is 'intel'  /(1/2)


P(W_0 | W) = ( P(W | W_0) * P(W_0) ) / P(W)

P('i7' | 'intel') = ??

### Example with more words
P('i7' | 'intel') = 1 * .33 / .33 = 1

|   | intel | i7  | 8700k |
|-------|-------|-----|-------|
| intel | 0     | 1   | 1     |
| i7    | 0     | 0   | 1     |
| 8700k | 0     | 0   | 0     |

P(W_0 | W) = Given the first word is 'intel', what is the probability the word after it will be 'i7'
P(W) = probability that a given word is 'intel' =
P(W_0) = probability that the next word is 'i7' =
P(W | W_0) = given that the next word is 'i7', what is the probability that the first is 'intel'


P(W_0 | W) = ( P(W | W_0) * P(W_0) ) / P(W)

